In [ ]:
# installing envs

from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
instructions1 = "You are a disciplined high-performance productivity coach. \
You believe success comes from structure, clarity, and execution. \
You create strategic, well-organized to-do lists and give serious, practical advice focused on long-term growth, consistency, and discipline."

instructions2 = "You are an energetic, inspiring motivational coach. \
You believe momentum comes from excitement and belief. \
You create engaging, empowering to-do lists and give uplifting advice that makes people feel capable, confident, and ready to take action immediately."

instructions3 = "You are a no-nonsense efficiency expert. \
You believe motivation follows action, not the other way around. \
You create concise, prioritized to-do lists and give direct, tactical advice that cuts distractions and focuses only on what truly moves the needle."

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [ ]:
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

# creating models, the point of this is too show how you can easily swap out different LLM providers and models with the same interface, and even use multiple models in the same agent if you want to (e.g. using a cheaper faster model for simple tasks and a more expensive powerful model for harder tasks)
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [ ]:
coach_1 = Agent(name="DeepSeek Productivity Coach", instructions=instructions1, model=deepseek_model)
coach_2 =  Agent(name="Gemini Motivational Coach", instructions=instructions2, model=gemini_model)
coach_3  = Agent(name="Llama3.3 Efficiency Expert",instructions=instructions3,model=llama3_3_model)

In [ ]:
## turn the agents into a list of tools

description = "Write a to-do list for the day based on the following input: {input}. / " \
"The to-do list should be focused on productivity and should be tailored to the user's needs and goals. " \
"Be sure to provide clear, actionable items that will help the user stay focused and motivated throughout the day." \
"Also make sure to include an appropriate real life quote to inspire the user based on the input. / " \

tool1 = coach_1.as_tool(tool_name="coach_1", tool_description=description)
tool2 = coach_2.as_tool(tool_name="coach_2", tool_description=description)
tool3 = coach_3.as_tool(tool_name="coach_3", tool_description=description)

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to myself """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("itzkotalo7@gmail.com")  # Change to your verified sender
    to_email = To("kotal008@umn.edu")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
subject_instructions = "You can write a subject for a motivation email. \
You are given a message and you need to write a subject for an email that is likely to make someone click on it."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a motivation email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [ ]:
email_tools = [subject_tool, html_tool, send_html_email]

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools, #using subject_tool & html_tool & the send_html_email tool as tools the agent can choose to use
    model="gpt-4o-mini", 
    handoff_description="Convert an email to HTML and send it")

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [ ]:
motivational_coach_instructions = """
You are a High-Performance Motivation Coach. Your goal is to create the single most effective Ultimate To-Do List using the three motivation_agent tools.

Follow these steps carefully:

1. Generate Draft Lists:
Use all three motivation_agent tools to generate three different versions of a to-do list:
- Strategy Agent (structured, long-term focused)
- Energy Agent (inspiring, momentum-driven)
- Execution Agent (concise, high-impact priorities)

Do not proceed until all three versions are generated.

2. Evaluate and Select:
Review the three lists and determine which one (or which combination) creates the most powerful and actionable Ultimate To-Do List.
You may call the agents multiple times if needed to refine the outputs.

3. Finalize and Handoff:
Create ONE final, optimized Ultimate To-Do List.
Pass ONLY the finalized list to the 'Productivity Coach' agent for logging or sending to Notion.

Crucial Rules:
- You must use the motivation_agent tools to generate all drafts — do not create the lists yourself.
- You must produce exactly ONE final consolidated to-do list.
- Do not pass multiple versions to the Productivity Coach — only the final optimized list.
"""


motivational_coach = Agent(
    name="Motivational Coach",
    instructions=motivational_coach_instructions,
    tools=tools, #first generate the to-do lists using the 3 coach tools
    handoffs=handoffs, # the handoff part is to the emailer_agent which uses the subject_tool and html_tool to create and send the email
    model="gpt-4o-mini")

message = """
Based on the current date and time, generate a powerful and slightly chaotic brain-dump of:
- urgent tasks I need to complete
- long-term goals I should be building toward
- small wins I can knock out quickly
- personal growth actions (health, learning, networking, mindset)
- one uncomfortable but high-reward action
- one fun or creative action

Do NOT overly organize it — let it feel like a motivated, ambitious mind mapping out everything that matters.

Then synthesize that mess into a prioritized “Start Here” section with the top 3 actions I should take immediately.

Finally, send the finalized version to Notion as today’s Ultimate Action Plan.
"""
with trace("Automated SDR"):
    result = await Runner.run(motivational_coach, message)

In [ ]:
from os import name


class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent_1 = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

class ExpletiveOutput(BaseModel):
    swear_in_msg: Dict[str, str]
    name: str

guardrail_agent_2 = Agent( 
    name="Expletive check",
    instructions="Check if the user has included any expletives in what they want you to do. These include swear words, curse words, or any other profane or offensive language.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)


In [ ]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent_1, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

@input_guardrail
async def guardrail_against_expletives(ctx, agent, message):
    result = await Runner.run(guardrail_agent_1, message, context=ctx.context)
    swear_in_msg = result.final_output.swear_in_msg
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=swear_in_msg)

In [ ]:
motivational_coach_guardlines = Agent(
    name="Motivational Coach with Guardrails",
    instructions=motivational_coach_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = """
Based on the current date and time, generate a powerful and slightly chaotic brain-dump of:
- urgent tasks I need to complete
- long-term goals I should be building toward
- small wins I can knock out quickly
- personal growth actions (health, learning, networking, mindset)
- one uncomfortable but high-reward action
- one fun or creative action

Do NOT overly organize it — let it feel like a motivated, ambitious mind mapping out everything that matters.

Then synthesize that mess into a prioritized “Start Here” section with the top 3 actions I should take immediately.

Finally, send the finalized version to Notion as today’s Ultimate Action Plan.
"""
with trace("Protected Automated SDR"):
    result = await Runner.run(motivational_coach_guardlines, message)

